@author: Timothy Chee Cheng Lui, https://github.com/timmehlui/

GeoChemNet is an interact tool for visualizing and interpreting outliers in geochemical data using networks. More details of the creation of the networks and protocols for use can be found in the accompanying paper, "Title" <https://doi>. (Work in progress)

GeoChemNet visualizes complex multidimensional data through a network/graph made of nodes and edges. The bipartite network is built from two kinds of nodes: elemental nodes for each element, and sample nodes for each analyte. Sample nodes are connected to elemental nodes using a spring, and the spring strength depends on concentration, i.e. a sample with very high Al will be pulled really close to the Al node. GeoChemNet aims to interpret the results of the equilibrium position of all these nodes after the balance of all the springs's pushing and pulling.

In [1]:
# Imports
from pyvis.network import Network
import networkx as nx
from networkx.algorithms import bipartite
import pandas as pd
import pyrolite.comp
import scipy
import matplotlib.pyplot as plt
from matplotlib import colors
from sklearn.preprocessing import MinMaxScaler
import matplotlib as mpl
import matplotlib.cm as cm
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from ipywidgets import interact, widgets

# QT is needed to create interactive visualizations in a pop-up window
%matplotlib qt 

In [2]:
### Important variables
## Please edit the following parameters accordingly

# File loctation and name in your computer
data_og = pd.read_csv("/Users/timmehlui/Documents/geochemicalData.csv")

# List of compositional elements you want to analyze in your dataset
element_list = ['Al', 'Cr', 'La', 'Nb', 'Sc', 'Sn', 'Ti', 'V', 'Zr'] #Example used in Demo 1
#element_list = ['V', 'Mo', 'Cr', 'Co', 'Ni', 'Pb', 'Zn', 'Sc', 'Zr', 'Sn', 'Ti', 'Nb'] #Figure 15
#element_list = ['U', 'V', 'Mo', 'Cr', 'Co', 'Ni', 'Cu', 'Pb', 'Zn', 'Al', 'La', 'Nb', 'Sc', 'Sn', 'Ti', 'Zr'] #Figure 16
element_list.sort()

# List of meta data that you still want to have access for, eg, Sample IDs, coordinate systems
meta_list = ['ID', 'east_rotate_zero', 'north_rotate_zero', 'cluster_Kobold', 'cluster']

# Name of the column that represents sample IDs
id_column_name = 'ID'

# Name of columns that represent X-Y coordinates (If using mapping tool)
x_coord = 'east_rotate_zero'
y_coord = 'north_rotate_zero'

# Threshold represents the cut off to determine what is or isn't an outlier. We recommend starting small and then consider decreasing.
# Optimal threshold value also depends on number of samples 
threshold = 0.85

# Parameters to tweak network distances.
weight_exponent = 1
k_multiplier = 1

# We offer two ways to create networks, each with pros and cons. For more details read the paper. Please un-comment your choice
network_style = 'scaling' # For scaling based thresholding
#network_style = 'percentile' # For percentile based thresholding

# If you want to focus on multivariate relations and remove the noise of univariate anomalies, make sure this is true
remove_univariate = True

# Random seed, in order for you to create consistent network representations
seed = 8


In order for this notebook to work we are going to need a cleaned and adequately formatted csv file. Please ensure that your file has the following properties.

- The data is in the form of a CSV file.
- All columns are titled.
- The data is complete, and there are now missing values, no empty cells, no non-numeric values.
- The data does not include any zeros. If adequate, consider replacing zeros for very small values.

In [ ]:
### Coming soon: Basic cleaning bot. Will ensure that the dataset provided matches the requirements for the program.

In [3]:
data_elems = data_og[element_list].copy()
data_meta = data_og[meta_list].copy()

# Turn data into CLR for proper treatment of compositional data
clr_data = data_elems.pyrocomp.CLR()
for name in clr_data.columns:
    new_name = name.split('(')[1].split('/')[0]
    dict_name = {name : new_name}
    clr_data = clr_data.rename(columns = dict_name)
    
# Standard scaling from 0-1 solves negatives issue and brings elements to equal relative weighting
scaler = MinMaxScaler()
data_clr = pd.DataFrame(scaler.fit_transform(clr_data), columns=clr_data.columns)

# Recombine and rename
data_recomb = pd.concat([data_meta, data_clr], axis=1)
data = data_recomb.copy()

# Create network based on thresholding style
G = nx.Graph()

if network_style == 'scaling':
    for i in data.itertuples():
        for j in range(len(meta_list), data.shape[1]):
            if i[j+1] >= threshold or i[j+1] <= 1 - threshold:
                G.add_edge(i[1], data.columns[j], weight = i[j+1] ** weight_exponent)
elif network_style == 'percentile':
    for j in range(len(meta_list), data.shape[1]):
        upper_limit = data[data.columns[j]].quantile(1-threshold)
        lower_limit = data[data.columns[j]].quantile(threshold)
        for i in data.itertuples():
            if i[j+1] >= upper_limit or i[j+1] <= lower_limit:
                G.add_edge(i[1], data.columns[j], weight = i[j+1] ** weight_exponent)
else:
    print('Network Style not found, please adjust')
    
# Ignore univariate relations. Loop over nodes, and if a node is only connected to one other node, remove.
if remove_univariate:
    nodes_to_remove = [node for node in G.nodes if G.degree(node) == 1]
    G.remove_nodes_from(nodes_to_remove)

pos = nx.spring_layout(G, seed = seed, k = k_multiplier * G.number_of_nodes() ** -0.5)

A great property of GeoChemNet is its ability to look at both geospatial and geochemical data at the same time! If you have x-y coordinates, consider using the following code block "Plotting Networks With A Map". If you don't have x-y coordinates, use the code block below.

In [6]:
################### Plotting Networks With A Map ###################
# Add text interactable for complex selecting
fig = plt.figure(figsize=(64,32))

def plot_variable(variable_color, select_for_text):
    elem_color = variable_color
    select_for_text = select_for_text
    
    # Process text
    if '.' in select_for_text:
        text_combos = select_for_text.split(',')
    
        plt.clf()
        
        data.loc[:, 'selected'] = 2

        count = 0
        
        ax1 = fig.add_axes([0.025, 0.1, 0.5, 0.8])  # add the top Axes

        vmin = 0.2
        vmax = 0.8
        cmap = cm.coolwarm
        
        if variable_color == 'cluster_Kobold':
            vmin = 0
            vmax = 9
            #cmap = cm.turbo # Good for differentiating colors
            cmap = cm.rainbow
            
        if variable_color == 'cluster':
            vmin = 0
            vmax = 6
            cmap = cm.turbo # Good for differentiating colors
            #cmap = cm.rainbow
        
        # Create network

        # Have to change color in geospatial plot
            
        for i in G.nodes():
            if type(i) == int:
                G.nodes[i]['attr'] = data.loc[data[id_column_name] == i, elem_color]

        color_map = []
        outline_map = []
        line_width_map = []
        labeldict = {}
        node_size_list = []
        norm = mpl.colors.Normalize(vmin = vmin, vmax=vmax)
        m = cm.ScalarMappable(norm = norm, cmap = cmap)

        for node in G:
            if type(node) == str:
                color_map.append('red')
                labeldict[node] = node
                node_size_list.append(2000)
                outline_map.append('black')
                line_width_map.append(0)
            else:
                color_map.append(mpl.colors.rgb2hex(m.to_rgba(G.nodes[node]['attr'])))
                node_size_list.append(100)
                outline_map.append('black')

                #### Change this line to choose selection
                # Strategy, assign all seelcted 1, and then slowly start turning things into 0

                for combo in text_combos:
                    highlow = combo.split()[0]
                    elem_select = combo.split()[1].split('.')[0]

                    if int(data.loc[data[id_column_name] == node, 'selected']) >= 1:
                        if highlow == 'high':
                            if G.has_edge(node, elem_select) and G.get_edge_data(elem_select, node)['weight'] ** (1/weight_exponent) > 0.5:
                                # Keep track in dataset
                                data.loc[data[id_column_name] == node, 'selected'] = 1
                            else:
                                data.loc[data[id_column_name] == node, 'selected'] = 0

                        if highlow == 'low':
                            if G.has_edge(node, elem_select) and G.get_edge_data(elem_select, node)['weight'] ** (1/weight_exponent) < 0.5:
                                # Keep track in dataset
                                data.loc[data[id_column_name] == node, 'selected'] = 1
                            else:
                                data.loc[data[id_column_name] == node, 'selected'] = 0

                if int(data.loc[data[id_column_name] == node, 'selected']) == 1:
                    line_width_map.append(2)
                else:
                    line_width_map.append(0)

        # Draw network
        nx.draw_networkx(G, pos = pos, node_color = color_map, edgecolors = outline_map, linewidths = line_width_map, labels = labeldict, node_size = node_size_list, with_labels = True, width = 0.1, font_size = 30)
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin = vmin, vmax=vmax))
        sm._A = []
        cbar = plt.colorbar(sm)
        tick_font_size = 15
        cbar.ax.tick_params(labelsize=tick_font_size)
        ax1.set_title('Network', fontsize=20)

        #Figure 2
        ax2 = fig.add_axes([0.575, 0.1, 0.3, 0.8])     # add the bottom Axes

        # Add selected attribute to dataset
        # Turn unselected data from nan to 0
        data['selected'] = data['selected'].fillna(0)

        # Draw map
        ax2.scatter(data[x_coord], data[y_coord], c = data[elem_color], cmap = cmap, norm=plt.Normalize(vmin = vmin, vmax=vmax), marker = 'o', s=20)
        ax2.scatter(data[data['selected'] == 1][x_coord], data[data['selected'] == 1][y_coord], facecolors = 'none', edgecolors='black', linewidth = 1, marker = 'o',  s=20)

        ax2.tick_params(axis='y', labelsize=15)
        ax2.tick_params(axis='x', labelsize=15)
        ax2.set_xlabel('Eastings (m)', fontsize=15)
        ax2.set_ylabel('Northings (m)', fontsize=15)
        ax2.legend(["Colored by \n" + elem_color , "Selected\nin Network"], fontsize = 15)
        #sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin = vmin, vmax=vmax))
        #sm._A = []
        #cbar = plt.colorbar(sm)
        #tick_font_size = 30
        #cbar.ax.tick_params(labelsize=tick_font_size)
        ax2.set_title('Geospatial Map', fontsize=20)

        fig.suptitle(' Network and Geospatial Map colored by ' + elem_color + ' selecting for ' + select_for_text, fontsize=22)

        # Want to zoom in to a specific part?
        # Uncomment the 2 lines below and select coordinates til the zoom is ideal
        #ax1.set_xlim((-0.4, 0.3))
        #ax1.set_ylim((-0.6, 1.05))
        
        
        plt.draw()
    
    #fig.show()

elem_dict = {}
element_list.append('cluster_Kobold')
element_list.append('cluster')
    
dropdown_color = widgets.Dropdown(options=element_list,
                            description='Choose variable to color by:',
                            layout={'width': 'max-content'},
                            style={'description_width': 'initial'})

selection_text = widgets.Textarea(value='high Al',
                                  description='What do you want to select for? Formatted "(high/low) Element":',
                                  layout={'width': 'max-content'},
                                  style={'description_width': 'initial'})

interact(plot_variable, variable_color=dropdown_color, select_for_text = selection_text)

interactive(children=(Dropdown(description='Choose variable to color by:', layout=Layout(width='max-content'),…

<function __main__.plot_variable(variable_color, select_for_text)>

Use the widgets above to interact with the code.
Use the drop down menu to color the network and map by specific attributes.
For the textbox, use it to select and highlight specific samples of interest.
It is crucial that you use the correct format. You can put as many statements as you want. They always have to include a
a) anomaly direction, either 'high' or 'low'
b) followed by a space bar ' '
c) followed by an element, with the same convention for your excel sheet, in our case capitalized 'Al' or 'Ti'
For example, you can input
'high Cr, low Ti, low Nb'

To make the figure plot, add a period to the end, to indicate "update figure".
To edit plot, remove the period first to avoid code errors.
Eg.
"high Cr, low Ti."
*figure will plot. Then remove period to edit selection*
"high Cr, low Ti"
*then update description and add period to plot*
"high Cr, low Ti, low Nb."
*new figure will plot highlighting the new selection*

In [ ]:
################### Plotting Networks (No Map Available) ###################
# Add text interactable for complex selecting
fig = plt.figure(figsize=(64,32))

def plot_variable(variable_color, select_for_text):
    elem_color = variable_color
    select_for_text = select_for_text
    
    # Process text
    if '.' in select_for_text:
        text_combos = select_for_text.split(',')
    
        plt.clf()
        
        data.loc[:, 'selected'] = 2

        count = 0
        
        ax1 = fig.add_axes([0.025, 0.1, 0.95, 0.8])  # add the top Axes

        vmin = 0.1
        vmax = 0.9
        cmap = cm.coolwarm

        # Have to change color in geospatial plot
            
        for i in G.nodes():
            if type(i) == int: # Find only the sample nodes, which are integers
                G.nodes[i]['attr'] = data.loc[data[id_column_name] == i, elem_color]

        color_map = []
        outline_map = []
        line_width_map = []
        labeldict = {}
        node_size_list = []
        norm = mpl.colors.Normalize(vmin = vmin, vmax=vmax)
        m = cm.ScalarMappable(norm = norm, cmap = cmap)

        for node in G:
            if type(node) == str:
                color_map.append('red')
                labeldict[node] = node
                node_size_list.append(2000)
                outline_map.append('black')
                line_width_map.append(0)
            else:
                color_map.append(mpl.colors.rgb2hex(m.to_rgba(G.nodes[node]['attr'])))
                node_size_list.append(100)
                outline_map.append('black')

                #### Change this line to choose selection
                # Strategy, assign all seelcted 1, and then slowly start turning things into 0

                for combo in text_combos:
                    highlow = combo.split()[0]
                    elem_select = combo.split()[1].split('.')[0]

                    if int(data.loc[data[id_column_name] == node, 'selected']) >= 1:
                        if highlow == 'high':
                            if G.has_edge(node, elem_select) and G.get_edge_data(elem_select, node)['weight'] ** (1/weight_exponent) > 0.5:
                                # Keep track in dataset
                                data.loc[data[id_column_name] == node, 'selected'] = 1
                            else:
                                data.loc[data[id_column_name] == node, 'selected'] = 0

                        if highlow == 'low':
                            if G.has_edge(node, elem_select) and G.get_edge_data(elem_select, node)['weight'] ** (1/weight_exponent) < 0.5:
                                # Keep track in dataset
                                data.loc[data[id_column_name] == node, 'selected'] = 1
                            else:
                                data.loc[data[id_column_name] == node, 'selected'] = 0

                if int(data.loc[data[id_column_name] == node, 'selected']) == 1:
                    line_width_map.append(2)
                else:
                    line_width_map.append(0)

        # Draw network
        nx.draw_networkx(G, pos = pos, node_color = color_map, edgecolors = outline_map, linewidths = line_width_map, labels = labeldict, node_size = node_size_list, with_labels = True, width = 0.1, font_size = 30)
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin = vmin, vmax=vmax))
        sm._A = []
        cbar = plt.colorbar(sm)
        tick_font_size = 15
        cbar.ax.tick_params(labelsize=tick_font_size)
        ax1.set_title('Network', fontsize=20)
        
        # Want to zoom in to a specific part?
        # Original
        ax1.set_xlim((-0.85, 0.85))
        ax1.set_ylim((-1, 1))
        
        fig.suptitle(' Network colored by ' + elem_color + ' selecting for ' + select_for_text, fontsize=30)

        plt.draw()
    
    # Reset data selected
    
    
    
    #fig.show()

elem_dict = {}
    
dropdown_color = widgets.Dropdown(options=element_list,
                            description='Choose variable to color by:',
                            layout={'width': 'max-content'},
                            style={'description_width': 'initial'})

selection_text = widgets.Textarea(value='high A',
                                  description='What do you want to select for? Formatted "(high/low) Element":',
                                  layout={'width': 'max-content'},
                                  style={'description_width': 'initial'})

interact(plot_variable, variable_color=dropdown_color, select_for_text = selection_text)